In [1]:
import matplotlib.pyplot as plt
import numpy as np

from snn_dpe.tools.network import (create_encoders, create_network,
                                   reset_network, run_network)
from snn_dpe.tools.plotting import plot_network, plot_spike_raster
from sympy.utilities.iterables import multiset_permutations
import copy
import time
from datetime import datetime
import json
from matplotlib.lines import Line2D

import tqdm
from multiprocessing import Pool

In [2]:
# measures the hamming distance between two spike rasters
def distance(sr1, sr2):
    total_distance = 0
    for n1, n2 in zip(sr1, sr2):
        for f1, f2 in zip(n1,n2):
            if f1 != f2:
                total_distance += 1

    return total_distance

In [ ]:
def run_test(connectivity):
    # create a test network and encoders
    n_neurons = 100
    n_synapses = int((n_neurons ** 2) * connectivity)

    sim_time = 100

    min_f = 1
    max_f = 100
    sim_f = 200

    n_enc = 5

    network_inputs = []
    network_responses = []

    np.random.seed()
    neurons = create_network(n_neurons, n_synapses, threshold_range=(1, 1), leak_range=(0.0, 0.0), weight_range=(1,1))

    encoders = create_encoders(n_enc, min_f, max_f, sim_f, enc_type='period')

    # create random encoder inputs from no activity to full activity
    enc_inputs = np.random.uniform(0, 1, (n_enc))

    # permute encoder order
        # this way we get a varying number of distances between each input, 
        # but the overall activity into the reservoir is the same for each run
    for enc_input in multiset_permutations(enc_inputs):
        fires = run_network(neurons, encoders, enc_input, sim_time)

        reset_network(neurons, encoders)

        network_responses.append(fires[:, n_enc:])
        network_inputs.append(fires[:, :n_enc])

    for i in range(len(network_inputs)):
        for j in range(i + 1, len(network_inputs)):
            input_d = distance(network_inputs[i], network_inputs[j]) /network_inputs[i].size
            response_d = distance(network_responses[i], network_responses[j]) /network_responses[i].size

            if input_d not in run['distances']:
                run['distances'][input_d] = []
                run['saturations'][input_d] = []

            run['distances'][input_d].append(response_d)
            run['saturations'][input_d].append(response_d)

